<a href="https://colab.research.google.com/github/NK2108/Deep-Learning/blob/main/emo_rec_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/samples/"
os.chdir(Root)
#load dataset and check list directory

In [16]:
import librosa
import soundfile as sf
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#import all necessary libraries

In [18]:
def extract_feature(file_name, mfcc, chroma, mel):
    with sf.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result
#Extract mfcc, chroma, mel from a sound file
#Soundfile can read and write sound files and represents audio data as NumPy arrays
#librosa.stft: Short-time Fourier transform, represents a signal in the time-frequency 
#domain by computing discrete Fourier transforms (DFT) over short overlapping windows
#librosa.feature.mfcc: peak loudness in decibels across all channels, we take the average
#librosa.feature.melspectrogram: computes a mel-scaled spectogram, logarithmically renders frequencies above a certain threshold


In [19]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['neutral', 'calm', 'happy','sad','angry', 'fearful', 'disgust', 'surprised']

In [24]:
#data load, using glob to implement "*" all wav files
def load_data(t_size):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/samples/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), np.array(y), test_size=t_size, random_state=9)

In [25]:
#Split the dataset
t_size=0.2
x_train,x_test,y_train,y_test=load_data(t_size)